1. 문제 정의: 챗봇의 필요성 및 주요 기능 정의
- 목적: 카카오테크부트캠프 수업 및 학습 도우미 챗봇
- 기능: 결석, 조퇴, 외출, 지각 횟수 누적 계산해서 알려주기; 수업 알림; 수업 관련 주요 링크 제공;
2. 솔루션 도출: 챗봇의 기능 구현 방법 및 API 활용 전략 도출
3. 설계: 챗봇의 아키텍처 및 주요 컴포넌트 설계
4. OpenAI API 키 발급 및 설정
5. 챗봇 프로젝트 초기 설정
6. 챗봇의 기본 대화 흐름 설계
7. 사용자 인증 및 권한 관리 기능 구현 < 사용자 이름, 교육과정 / 권한 관리 
8. 교육생 질문 응답 기능 구현 < 질문 자연어 처리 필요
9. 일정 관리 기능 구현 < 투두처럼 일정 기록 기능, 수업 출결 
10. 과제 제출 안내 기능 구현 < 하루 학습 계획 등록, 보고서 링크나 파일 제출하면 달성으로 표기 
11. OpenAI API 연동 및 테스트
12. 챗봇의 자연어 처리 기능 테스트 및 개선
13. 챗봇 UI/UX 디자인 및 구현
14. 챗봇의 응답 성능 최적화
15. 챗봇 배포 및 호스팅
16. 배포된 챗봇의 동작 확인 영상 제작
17. 최종 결과물 제출 (챗봇 소스 코드, 실행 파일 또는 URL, 시연 영상, 사용 설명서)

In [2]:
import discord
from discord.ext import commands
import json
import nest_asyncio

nest_asyncio.apply()

intents = discord.Intents.default()
intents.message_content = True  # 메시지 내용을 읽을 수 있도록 설정

bot = commands.Bot(command_prefix='! ', intents=intents)


################# 기능 1: 출결 관리 ################
## 기능 1-1: 사용자마다 출결 확인 - 사용자 이름과 교육 과정으로 구분 ##

attendance_file = 'attendance.json'

def load_attendance():
    try:
        with open(attendance_file, 'r') as f:
            data = json.load(f)
            # Ensure that all categories are lists
            for key in data.keys():
                if not isinstance(data[key], list):
                    data[key] = []
            return data
    except FileNotFoundError:
        return {"결석": [], "조퇴": [], "외출": [], "지각": []}

def save_attendance(data):
    with open(attendance_file, 'w') as f:
        json.dump(data, f)

attendance = load_attendance()

# 조퇴,외출,지각 카운트는 계속 누적 / 결석 += 조외지 합계 //3 시도해보기 

@bot.command(name='출석관리')
async def manage_attendance(ctx, date: str, category: str):
    try:
        print(f'Received command 출석관리 with date={date} and category={category}')  # 디버깅 출력
        if category not in attendance:
            await ctx.send(f"잘못된 카테고리입니다: {category}")
            return

        attendance[category].append(date)
        save_attendance(attendance)
        await ctx.send(f"{date} {category} 등록되었습니다.")
    except Exception as e:
        print(f"Error in manage_attendance: {e}")
        await ctx.send(f"Error: {e}")

@bot.command(name='횟수')
async def count_attendance(ctx, category: str):
    try:
        print(f'Received command 횟수 with category={category}')  # 디버깅 출력
        if category in attendance:
            count = len(attendance[category])
            await ctx.send(f"{category} 횟수: {count}")
        else:
            await ctx.send(f"잘못된 카테고리입니다: {category}")
    except Exception as e:
        print(f"Error in count_attendance: {e}")
        await ctx.send(f"Error: {e}")

@bot.command(name='날짜')
async def dates_attendance(ctx, category: str):
    try:
        print(f'Received command 날짜 with category={category}')  # 디버깅 출력
        if category in attendance:
            dates = attendance[category]
            if dates:
                await ctx.send(f"{category} 날짜: {', '.join(dates)}")
            else:
                await ctx.send(f"{category} 기록이 없습니다.")
        else:
            await ctx.send(f"잘못된 카테고리입니다: {category}")
    except Exception as e:
        print(f"Error in dates_attendance: {e}")
        await ctx.send(f"Error: {e}")

@bot.command(name='초기화')
async def reset_count_attendance(ctx, category: str):
    try:
        if category in attendance:
            attendance[category] = []
            save_attendance(attendance)
            await ctx.send(f"{category} 기록이 초기화되었습니다.")
        else:
            await ctx.send(f"잘못된 카테고리입니다: {category}")
    except Exception as e:
        print(f"Error in reset_count_attendance: {e}")
        await ctx.send(f"Error: {e}")

@bot.command(name='명령어')
async def show_commands(ctx):
    commands_list = (
        "!출석관리 <날짜> <카테고리> - 특정 날짜에 해당 카테고리를 기록합니다.\n"
        "!횟수 <카테고리> - 지정된 카테고리의 누적 횟수를 출력합니다.\n"
        "!날짜 <카테고리> - 지정된 카테고리의 모든 날짜 기록을 출력합니다.\n"
    )
    await ctx.send(commands_list)


################ 기능 2: 수업 알림 ################

#### 기능 2-1: 일정 관리 구현(캘린더, 토요일 수업 등록, 삭제 기능 ####
#### 기능 2-2: 개인 일정 등록 - 사용자 이름, 교육과정으로 구분 가능 ####



############# 기능 3: 수업 관련 링크 연결 ##############



### 기능 4: 과제 제출(학습 계획 등록 > 링크나 파일 제출 시 학습 계획 완료) ###


########## 기능 5: 질문 응답 기능 구현 ############

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')

async def main():
    async with bot:
        await bot.start(TOKEN)

if __name__ == '__main__':
    import asyncio
    asyncio.run(main())


Logged in as manage#6062
Received command 출석관리 with date=7/13 and category=결석
Received command 횟수 with category=결석
Received command 날짜 with category=결석
